In [1]:
import pandas as pd
import os
import mysql.connector
import warnings
warnings.filterwarnings("ignore")

path = os.getcwd() + "/csv_data"

## Carregando os dados já tratados

In [2]:
# é necessário separar em duas partes 
# as tabelas "principais" precisam ser adicionadas antes para preservar a ordem de inserção
# do contrário, se a ordem alfabética, há erro de integridade
df_paths = [file.split(".csv")[0] for file in os.listdir(path) if ".csv" in file and "_" not in file]
df_paths += [file.split(".csv")[0] for file in os.listdir(path) if ".csv" in file and "_" in file]
df_paths

['genre',
 'rating',
 'actor',
 'director',
 'content',
 'type',
 'content_cast',
 'content_rating',
 'content_type',
 'content_director',
 'content_genre',
 'movie_duration',
 'tv_show_duration']

In [3]:
dataframes = {}
for df in df_paths:
    dataframes[df] = pd.read_csv(f"{path}/{df}.csv")

## Enviando os dados para o MySQL

In [4]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Pedro051220011-",
    database="netflix"
)

cursor = conn.cursor()

In [5]:
# para cada dataframe, há uma tabela correspondente
# iremos iterar por cada um dos dataframes que abrimos
for df in dataframes.keys():
    exception = False
    # estando em um dataframe, iremos iterar por todas as suas linhas
    # as linhas serão também as linhas na tabela correspondente no MySQL
    for idx, row in dataframes[df].iterrows():
        data = ""
        for value in row.values:
            if pd.isna(value):
                data += "NULL, "
            elif isinstance(value, str):
                value = value.replace("'", "")
                data += f"'{value}', "
            else:
                data += f"{value}, "
        data = data[:-2]
        
        insert_query_str = "INSERT INTO {} VALUES ({})".format(df, data)

        # caso alguma inserção dê erro, interrompemos a execução para ver o que aconteceu
        try:
            cursor.execute(insert_query_str)
        except Exception as e:
            print(insert_query_str)
            print(e)
            exception = True
            break 
    if exception:
        break

# só escrevemos as mudanças no banco se todas as inserções foram bem sucedidas
if not exception:
    conn.commit()